# Importing Neccessary Modules

In [ ]:
import cv2
import pyautogui
import speech_recognition as sr
import threading
from ultralytics import YOLO
import streamlit as st
import webbrowser
import time


# Main_Code

In [ ]:

# Load your custom-trained YOLOv8 model

model= YOLO(r'C:\Users\Administrator\Desktop\final_custom_model\best.pt')

# Define gesture-key mappings for controlling the game
gesture_key_map = {
    'thumbsup': 'up',
    'thumbsdown': 'down',
    'thumbsleft': 'left',
    'thumbsright': 'right',
    # Add other mappings as needed
}

# Define speech command mappings
speech_command_map = {
    'start': 'x',
    'stop': 'y',
    
    # Add more commands as needed
}

# Global variable to store detected gestures
current_gesture = None

# Function to recognize speech in real-time
def recognize_speech():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    while True:
        with mic as source:
            recognizer.adjust_for_ambient_noise(source)
            print("Listening for commands...")
            audio = recognizer.listen(source)

        try:
            command = recognizer.recognize_google(audio).lower().strip()
            print(f"Recognized command: {command}")
            if command in speech_command_map:
                pyautogui.press(speech_command_map[command])
                print(f"Executed command: {command}")
        except sr.UnknownValueError:
            print("Sorry, I did not understand that.")
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")

# Function to detect gestures
def detect_gestures(frame):
    global current_gesture
    results = model(frame)
    detected_gestures = set()

    for result in results:
        for bbox in result.boxes:
            class_id = int(bbox.cls)
            object_class = model.names[class_id]
            confidence = bbox.conf

            # Check if the confidence level is above a threshold (e.g., 0.5)
            if confidence > 0.2:
                x1, y1, x2, y2 = map(int, bbox.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, object_class, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

                if object_class in gesture_key_map:
                    detected_gestures.add(object_class)

    if detected_gestures:
        new_gesture = detected_gestures.pop()
        if new_gesture != current_gesture:
            current_gesture = new_gesture
            print(f"Detected gesture: {current_gesture}, pressing key: {gesture_key_map[current_gesture]}")
            pyautogui.press(gesture_key_map[current_gesture])
    else:
        current_gesture = None

    return frame

# Function to run the game control loop
def run_game_control():
    global current_gesture

    # Open a connection to the webcam
    cap = cv2.VideoCapture(0)

    # Run the speech recognition in a separate thread
    speech_thread = threading.Thread(target=recognize_speech, daemon=True)
    speech_thread.start()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = detect_gestures(frame)

        # Display the frame with detections
        cv2.imshow('Game Control', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Streamlit UI
st.title("Gesture-Controlled Game Player")
st.write("Choose a game to play using gestures and voice commands.")

# List of games (Add more links as needed)
games = {
    "Tetris": "https://tetris.com/play-tetris",
    "Pac-Man": "https://g.co/kgs/9kYA8Cm",
    "Snake": "https://playsnake.org/",
    "Minesweeper": "https://minesweeper.online/"
}

game_choice = st.selectbox("Select a game", list(games.keys()))

if st.button("Start Game"):
    game_url = games[game_choice]
    webbrowser.open(game_url)  # Open the selected game in a new browser tab
    st.write(f"Game started: {game_choice}")

    # Run the game control function in a separate thread to keep the Streamlit UI responsive
    control_thread = threading.Thread(target=run_game_control, daemon=True)
    control_thread.start()

# Add a placeholder for displaying the webcam feed (this will be shown in a separate window by OpenCV)
st.write("Webcam feed will be displayed in a separate window.")

